In [1]:
!pip install minsearch


[notice] A new release of pip available: 22.3 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [8]:
import minsearch

In [9]:
import json

In [10]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [11]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [12]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [13]:
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

SELECT * WHERE course = 'data-engineering-zoomcamp';

In [14]:
q = 'the course has already started, can I still enroll?'

In [15]:
index.fit(documents)

In [ ]:
import os
import requests

HF_TOKEN=""

def ask(question):
    API_URL = "https://router.huggingface.co/novita/v3/openai/chat/completions"
    headers = {"Authorization": f"Bearer {HF_TOKEN}"}
    payload = {
        "messages": [
            {
                "role": "user",
                "content": question
            }
        ],
        "model": "deepseek/deepseek-v3-0324",
    }

    response = requests.post(API_URL, headers=headers, json=payload)
    print(response.json()["choices"][0]["message"])
    return response.json()["choices"][0]["message"]

In [17]:
response = client.chat.completions.create(
    model='gpt-4o',
    messages=[{"role": "user", "content": q}]
)

response.choices[0].message.content

NameError: name 'client' is not defined

In [18]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )

    return results

In [19]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [14]:
def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [20]:
query = 'how do I run kafka?'

def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = ask(prompt)
    return answer

In [21]:
rag(query)

{'role': 'assistant', 'content': 'To run Kafka, follow the appropriate instructions based on your use case:\n\n1. **For Java Kafka (e.g., running a producer/consumer/KStreams in terminal):**  \n   Navigate to your project directory and execute:  \n   ```bash\n   java -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java\n   ```\n\n2. **For Python Kafka (if encountering "Module \'kafka\' not found"):**  \n   Create and activate a virtual environment, then install dependencies:  \n   ```bash\n   python -m venv env\n   source env/bin/activate  # On Windows: env\\Scripts\\activate\n   pip install -r ../requirements.txt\n   ```\n   Alternatively, if facing issues with `kafka-python`, use:  \n   ```bash\n   pip install kafka-python-ng\n   ```\n\n3. **For permission issues (e.g., `./build.sh: Permission denied`):**  \n   Run:  \n   ```bash\n   chmod +x build.sh\n   ```\n\n4. **Ensure Docker containers are running** if your setup depends on them.  \n\nFor P

{'role': 'assistant',
 'content': 'To run Kafka, follow the appropriate instructions based on your use case:\n\n1. **For Java Kafka (e.g., running a producer/consumer/KStreams in terminal):**  \n   Navigate to your project directory and execute:  \n   ```bash\n   java -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java\n   ```\n\n2. **For Python Kafka (if encountering "Module \'kafka\' not found"):**  \n   Create and activate a virtual environment, then install dependencies:  \n   ```bash\n   python -m venv env\n   source env/bin/activate  # On Windows: env\\Scripts\\activate\n   pip install -r ../requirements.txt\n   ```\n   Alternatively, if facing issues with `kafka-python`, use:  \n   ```bash\n   pip install kafka-python-ng\n   ```\n\n3. **For permission issues (e.g., `./build.sh: Permission denied`):**  \n   Run:  \n   ```bash\n   chmod +x build.sh\n   ```\n\n4. **Ensure Docker containers are running** if your setup depends on them.  \n\nFor 

In [22]:
rag('the course has already started, can I still enroll?')

{'role': 'assistant', 'content': "Yes, you can still enroll in the course even after it has started. You are eligible to submit homeworks, but be mindful of the deadlines for final projects. The course materials will remain available after the course finishes, allowing you to follow at your own pace. \n\nFor more details, you can join the course Telegram channel and register in DataTalks.Club's Slack to stay updated and ask questions if needed."}


{'role': 'assistant',
 'content': "Yes, you can still enroll in the course even after it has started. You are eligible to submit homeworks, but be mindful of the deadlines for final projects. The course materials will remain available after the course finishes, allowing you to follow at your own pace. \n\nFor more details, you can join the course Telegram channel and register in DataTalks.Club's Slack to stay updated and ask questions if needed."}

In [23]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [24]:
from elasticsearch import Elasticsearch

In [25]:
es_client = Elasticsearch('http://localhost:9200') 

In [26]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

BadRequestError: BadRequestError(400, 'resource_already_exists_exception', 'index [course-questions/nJnNqrKmSsKW0w8qu3N6Vw] already exists')

In [27]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [28]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [29]:
from tqdm.auto import tqdm

In [30]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

In [31]:
query = 'I just disovered the course. Can I still join it?'

In [32]:
def elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs

In [33]:
def rag(query):
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    answer = ask(prompt)
    return answer

In [34]:
# rag(query)
elastic_search( "How do copy a file to a Docker container?")

[{'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I debug a docker container?',
  'course': 'machine-learning-zoomcamp'},
 {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I debug a docker container?',
  'course': 'machine-learning-zoomcamp'},
 {'text': 'Launch the container im

In [35]:
context_template = """
Q: {question}
A: {text}
""".strip()

q4_response = elastic_search( "How do copy a file to a Docker container?")

In [36]:
context_template = []

for i in q4_response:
    context_template.append(f"""
Q: {i['question']}
A: {i['text']}\n\n
""".strip())    
print(len(context_template))

5


In [37]:
question="How do copy a file to a Docker container?"
context=context_template
prompt_template = f"""
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

print(prompt_template)
print(len(prompt_template))


You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: How do copy a file to a Docker container?

CONTEXT:
['Q: How do I debug a docker container?\nA: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)', 'Q: How do I debug a docker container?\nA: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)', 'Q: How do I debug a docker contain

In [42]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/DeepSeek-V3-0324")

# Your input text
text = "Hello, how are you doing today?"

# Tokenize the text
tokens = tokenizer.encode(prompt_template, add_special_tokens=True)

# Count tokens
print(f"Token count: {len(tokens)}")


Token count: 522
